# Exploring the links between Endocrine disruptors and Coronary Heart Disease through machine learning

## Group 4

Run the immediate next chunk of code once per session (whenever session expires or you close the tab)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting path to our group project folder where the data is

In [3]:
import os

base_path = '/content/drive/My Drive/Group 4'

Code below will verify if the base path was set correctly (it should list the files within our group folder)

['Group4 AIcolab Project.ipynb', 'NHANES 2013-2014 data', 'NHANES 2015-2016 data', 'NHANES 2017-2018 data', 'Group 4 Project Presentation- July 3rd.gslides', 'Project update-July 10th.gslides', 'Data Dictionary.gsheet', 'EDA.ipynb', 'EDA results.gdoc', '7 17 25 Group 4 Update.gslides', 'Cluster.ipynb']


The next code will load and merge the files from each year

In [5]:
import pandas as pd

def load_xpt_file(cycle_folder, filename):
    full_path = os.path.join(base_path, cycle_folder, filename)
    return pd.read_sas(full_path)


file_map = {
    'NHANES 2013-2014 data': {
        'demo': 'DEMO_H.xpt',
        'mcq': 'MCQ_H.xpt',
        'bp': 'BPX_H.xpt',
        'chol': 'TCHOL_H.xpt',
        'hdl': 'HDL_H.xpt',
        'phthalates': 'PHTHTE_H.xpt',
        'pfas': 'PFAS_H.xpt'
    },
    'NHANES 2015-2016 data': {
        'demo': 'DEMO_I.xpt',
        'mcq': 'MCQ_I.xpt',
        'bp': 'BPX_I.xpt',
        'chol': 'TCHOL_I.xpt',
        'hdl': 'HDL_I.xpt',
        'phthalates': 'PHTHTE_I.xpt',
        'pfas': 'PFAS_I.xpt'
    },
    'NHANES 2017-2018 data': {
        'demo': 'DEMO_J.xpt',
        'mcq': 'MCQ_J.xpt',
        'bp': 'BPX_J.xpt',
        'chol': 'TCHOL_J.xpt',
        'hdl': 'HDL_J.xpt',
        'phthalates': 'PHTHTE_J.xpt',
        'pfas': 'PFAS_J.xpt'
    }
}

In [6]:
merged_list = []

for cycle, files in file_map.items():
    print(f"Processing {cycle}...")

    try:
        demo = load_xpt_file(cycle, files['demo'])
        mcq = load_xpt_file(cycle, files['mcq'])
        bp = load_xpt_file(cycle, files['bp'])
        chol = load_xpt_file(cycle, files['chol'])
        hdl = load_xpt_file(cycle, files['hdl'])
        phthalates = load_xpt_file(cycle, files['phthalates'])
        pfas = load_xpt_file(cycle, files['pfas'])

        merged = demo.merge(mcq, on='SEQN', how='inner') \
                     .merge(bp, on='SEQN', how='inner') \
                     .merge(chol, on='SEQN', how='inner') \
                     .merge(hdl, on='SEQN', how='inner') \
                     .merge(phthalates, on='SEQN', how='inner') \
                     .merge(pfas, on='SEQN', how='inner')

        merged_list.append(merged)
        print(f"{cycle} merged: {merged.shape}")

    except Exception as e:
        print(f"Error processing {cycle}: {e}")


Processing NHANES 2013-2014 data...
NHANES 2013-2014 data merged: (2339, 212)
Processing NHANES 2015-2016 data...
NHANES 2015-2016 data merged: (2170, 214)
Processing NHANES 2017-2018 data...
NHANES 2017-2018 data merged: (2133, 203)


We can see from the above code the shape of the merged data for each cycle(2 year period). As mentioned in our presentation July 3rd, the 2013-2014 cycle has 2339 observations and 212 variables/features. 2015-2016 has 2170 observations and 214 features. Lastly 2017-2018 has 2133 observations and 203 features.

In [7]:
# Check which columns are shared by all three merged DataFrames
common_cols = set.intersection(*[set(df.columns) for df in merged_list])
print(f"Common columns across all cycles: {len(common_cols)}")

#See what columns are missing or extra in each cycle
for i, df in enumerate(merged_list):
    missing = common_cols.symmetric_difference(set(df.columns))
    print(f"Cycle {i+1} missing or extra columns: {missing}")

Common columns across all cycles: 142
Cycle 1 missing or extra columns: {'MCQ240G', 'MCQ240L', 'MCQ180K', 'LBDPFDOL', 'MCQ240O', 'MCQ380', 'MCQ075', 'MCQ240DD', 'LBXPFBS', 'MCQ240T', 'URXUCR', 'DMDHREDU', 'MCQ240S', 'MCQ370A', 'MCQ180N', 'MCQ370B', 'MCQ365B', 'MCQ180F', 'MCQ240I', 'MCQ240Y', 'MCQ180A', 'MCQ180B', 'MCQ240X', 'MCQ180C', 'MCQ086', 'MCQ240F', 'PEASCTM1', 'MCQ180L', 'MCQ180G', 'MCQ365C', 'MCQ240U', 'LBDPFBSL', 'MCQ365A', 'MCQ180D', 'LBXPFDO', 'MCQ240C', 'MCQ240D', 'MCQ240CC', 'PEASCST1', 'MCQ240A', 'MCQ084', 'MCQ240H', 'MCQ240BB', 'MCQ240Q', 'MCQ240K', 'DMDHRAGE', 'MCQ240E', 'MCQ240N', 'MCQ240V', 'MCQ240DK', 'MCQ070', 'MCQ180M', 'MCQ240R', 'MCQ370D', 'MCQ370C', 'MCQ365D', 'MCQ240W', 'MCQ240Z', 'MCQ082', 'DMDHRBR4', 'MCQ240AA', 'MCQ240M', 'MCQ180E', 'LBDPFHPL', 'DMDHSEDU', 'LBXPFHP', 'MCQ240P', 'DMDHRMAR', 'MCQ240J', 'MCQ240B'}
Cycle 2 missing or extra columns: {'MCQ240G', 'MCQ240L', 'MCQ180K', 'LBDPFDOL', 'MCQ240O', 'MCQ240DD', 'MCQ240T', 'DMDHREDU', 'LBXNFOA', 'OSQ230', 'L

# **COMMON COLUMNS FOR DATA DICTIONARY**

In [ ]:
#All the common columns
print(common_cols)
#How many common columns
print(len(common_cols))

{'MCQ160D', 'MCQ025', 'URXCOP', 'MCQ010', 'RIDEXAGM', 'MCQ170L', 'MCQ160F', 'MCQ203', 'URXMZP', 'URDMNPLC', 'SIALANG', 'MCQ220', 'LBXPFUA', 'MCQ092', 'BPXML1', 'BPXPTY', 'URDCNPLC', 'URDMIBLC', 'MCQ230B', 'BPXSY4', 'URXMEP', 'MCQ053', 'RIDSTATR', 'MCD093', 'URXMNP', 'MIALANG', 'RIDRETH3', 'AIALANGA', 'URDCOPLC', 'PEASCCT1', 'MCQ170M', 'BPXSY1', 'LBDPFHSL', 'SIAINTRP', 'URXCNP', 'DMDCITZN', 'RIDAGEYR', 'URDMEPLC', 'MCQ080', 'BPXPLS', 'DMDHHSIZ', 'MCQ230A', 'LBXPFDE', 'URDMOHLC', 'RIDAGEMN', 'FIAINTRP', 'URDMHPLC', 'URDMZPLC', 'MCQ050', 'AGQ030', 'URDMBPLC', 'RIDRETH1', 'DMDHRGND', 'URXECP', 'MCQ160K', 'INDFMIN2', 'BPXDI1', 'INDFMPIR', 'MCQ230D', 'URDECPLC', 'MCQ206', 'LBDPFNAL', 'FIAPROXY', 'RIDEXPRG', 'MCQ230C', 'LBDTCSI', 'BPAARM', 'DMQADFC', 'URDMCHLC', 'DMDYRSUS', 'FIALANG', 'SEQN', 'DMDHHSZA', 'BPXDI4', 'MCQ170K', 'BPXSY2', 'RIDEXMON', 'SDMVPSU', 'DMDHHSZE', 'MCQ149', 'MCQ160C', 'RIAGENDR', 'SDDSRVYR', 'URXMOH', 'SIAPROXY', 'MIAPROXY', 'BPXDI3', 'URXMHP', 'DMDHHSZB', 'BPXPULS', 'MC

# Merging the cycles

In [8]:
#Code to merge the data
combined_df = pd.concat(merged_list, ignore_index=True)

#Code to see how many observations and features are in the merged dataset
print("NHANES 2013-2018 shape", combined_df.shape)

NHANES 2013-2018 shape (6642, 273)


# Target variable

Our target variable will probably be MCQ160c which is from the questionnaire data for each cycle. The variable represents: "Has a doctor or other health professional ever told {you/SP} that {you/s/he} . . .had coronary heart disease

# **Random Seed**

In [ ]:
random_seed = 42

import numpy as np
np.random.seed(random_seed)

#Split the data

In [ ]:
#Reminder of how many observations and features our initial dataset has
combined_df.shape

(6642, 273)

In [ ]:
#We will need a data dictionary of all the columns in the combined dataset
print(combined_df.columns)

Index(['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN',
       'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'RIDEXAGM',
       ...
       'MCQ371A', 'MCQ371B', 'MCQ371C', 'MCQ371D', 'URXECPT', 'URDEPCLC',
       'URXMHHT', 'URDMHTLC', 'URXMONP', 'URDMONLC'],
      dtype='object', length=273)


## Step 1 after loading dataset: Split the data

In [ ]:
#Target variable has missing values that need to be handled before splitting the data

#Creating a raw copy to preserve the initial data

combined_df_raw = combined_df.copy(deep = True)

print(combined_df_raw['MCQ160C'].isnull().sum())

1263


In [ ]:
#Drop nan in target variables
combined_df_raw = combined_df_raw.dropna(subset = ['MCQ160C'])

#Check the null values of target variable
print(combined_df_raw['MCQ160C'].isnull().sum())

#Check distribution of target variable
print(combined_df_raw['MCQ160C'].value_counts())



0
MCQ160C
2.0    5142
1.0     221
9.0      16
Name: count, dtype: int64


In [ ]:
#Remove everything but 1 and 2 from the target
combined_df_raw = combined_df_raw[combined_df_raw['MCQ160C'].isin([1, 2])]

#Map 1 and 2 as 1 and 0:
combined_df_raw['MCQ160C'] = combined_df_raw['MCQ160C'].map({1: 1, 2: 0})

#Check if the mapping worked
print(combined_df_raw['MCQ160C'].value_counts())

MCQ160C
0    5142
1     221
Name: count, dtype: int64


# **Combined Features Below**

In [ ]:
print(combined_df_raw.columns.tolist())

['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGE', 'DMDHRBR4', 'DMDHREDU', 'DMDHRMAR', 'DMDHSEDU', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR', 'MCQ010', 'MCQ025', 'MCQ035', 'MCQ040', 'MCQ050', 'AGQ030', 'MCQ053', 'MCQ070', 'MCQ075', 'MCQ080', 'MCQ082', 'MCQ084', 'MCQ086', 'MCQ092', 'MCD093', 'MCQ149', 'MCQ151', 'MCQ160A', 'MCQ180A', 'MCQ195', 'MCQ160N', 'MCQ180N', 'MCQ160B', 'MCQ180B', 'MCQ160C', 'MCQ180C', 'MCQ160D', 'MCQ180D', 'MCQ160E', 'MCQ180E', 'MCQ160F', 'MCQ180F', 'MCQ160G', 'MCQ180G', 'MCQ160M', 'MCQ170M', 'MCQ180M', 'MCQ160K', 'MCQ170K', 'MCQ180K', 'MCQ160L', 'MCQ170L', 

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
df_train, df_test = train_test_split(combined_df_raw, train_size = 0.6, random_state = random_seed, stratify = combined_df_raw['MCQ160C'])

df_val, df_test = train_test_split(df_test, train_size = 0.5, random_state = random_seed, stratify = df_test['MCQ160C'])

df_train, df_val, df_test = df_train.reset_index(drop = True), df_val.reset_index(drop = True), df_test.reset_index(drop = True)

In [ ]:
#Filter for common features and target
features_to_keep = list(common_cols)

df_train = df_train[features_to_keep]
df_val = df_val[features_to_keep]
df_test = df_test[features_to_keep]

In [ ]:
#Check the shape or dimension of df_train
df_train.shape

(3217, 142)

In [ ]:
#Check the shape of df_val
df_val.shape

(1073, 142)

In [ ]:
#Check shape of df_test
df_test.shape

(1073, 142)

In [ ]:
#Check if target is in the right place
print(df_train.columns.unique())
'MCQ160C' in common_cols

Index(['URDCOPLC', 'LBDPFHSL', 'MCQ230B', 'LBXMPAH', 'LBDPFUAL', 'MCQ010',
       'DMDFMSIZ', 'SDMVPSU', 'DMDEDUC3', 'MCQ160O',
       ...
       'BPXDI3', 'BPXDI2', 'LBDPFNAL', 'RIAGENDR', 'DMDBORN4', 'WTSB2YR_y',
       'URXMHP', 'URXCNP', 'MCQ151', 'MCD093'],
      dtype='object', length=142)


True

## Step 2: Handle uncommon features in df_train, df_test, df_val

In [ ]:
train_cols = set(df_train.columns)
val_cols = set(df_val.columns)
test_cols = set(df_test.columns)

common_columns = train_cols & val_cols & test_cols
train_only = train_cols - common_columns
val_only = val_cols - common_columns
test_only = test_cols - common_columns

print("Train_only features:", train_only)
print("Val_only features:", val_only)
print("Test_only features:", test_only)

Train_only features: set()
Val_only features: set()
Test_only features: set()


# Step 3: Handling/removing identifiers

In [ ]:
#Combine train, val, test again before checking all for identifiers
df = pd.concat([df_train, df_val, df_test], sort = False)

In [ ]:
#Identify any identifiers
'''We can find identifiers through to possible means: checking if the data type for a column is a float(identifiers typically are not), and checking if each value for the column is unique'''

def id_checker(df):
  return df[[var for var in df.columns
             if df[var].nunique(dropna= True) == df[var].notnull().sum()]]

#Check identifiers
df_id = id_checker(df)
df_id.head()
df_id.shape

(5363, 8)

In [ ]:
#Check the indentifiers
print(df_id.columns)
for col in df_id.columns:
    print(f"\n{col}")
    print("Unique values:", df[col].nunique())
    print("Non-null values:", df[col].notnull().sum())
    print("Sample values:", df[col].dropna().unique()[:10])


Index(['DMDEDUC3', 'SEQN', 'BPXCHR', 'MCQ149', 'MCQ230D', 'RIDEXAGM',
       'RIDAGEMN', 'MCQ151'],
      dtype='object')

DMDEDUC3
Unique values: 0
Non-null values: 0
Sample values: []

SEQN
Unique values: 5363
Non-null values: 5363
Sample values: [96121. 93758. 97619. 74028. 87987. 83539. 94698. 80245. 98336. 97109.]

BPXCHR
Unique values: 0
Non-null values: 0
Sample values: []

MCQ149
Unique values: 0
Non-null values: 0
Sample values: []

MCQ230D
Unique values: 1
Non-null values: 1
Sample values: [66.]

RIDEXAGM
Unique values: 0
Non-null values: 0
Sample values: []

RIDAGEMN
Unique values: 0
Non-null values: 0
Sample values: []

MCQ151
Unique values: 0
Non-null values: 0
Sample values: []


In [ ]:
import numpy as np

# Remove identifiers from df_train
df_train.drop(columns=np.intersect1d(df_id.columns, df_train.columns), inplace=True)

# Remove identifiers from df_val
df_val.drop(columns=np.intersect1d(df_id.columns, df_val.columns), inplace=True)

# Remove identifiers from df_test
df_test.drop(columns=np.intersect1d(df_id.columns, df_test.columns), inplace=True)

#Check shape of df after dropping columns
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3217, 134)
(1073, 134)
(1073, 134)


/tmp/ipython-input-27-3195956836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=np.intersect1d(df_id.columns, df_train.columns), inplace=True)
/tmp/ipython-input-27-3195956836.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.drop(columns=np.intersect1d(df_id.columns, df_val.columns), inplace=True)
/tmp/ipython-input-27-3195956836.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(colu

# Step 4: Handling date time variables

In [ ]:
#Making a cycle variable for the data
print(df['SDDSRVYR'].value_counts().sort_index())

cycle_map = {
    8.0: '2013-2014',
    9.0: '2015-2016',
    10.0: '2017-2018'
}

SDDSRVYR
8.0     1857
9.0     1745
10.0    1761
Name: count, dtype: int64


In [ ]:
#Combine train, test, val again to check for date time variables
df = pd.concat([df_train, df_val, df_test], sort = False)

#Check data types
print(df.dtypes.unique())

#Check if we have date time variables
non_numeric = df.select_dtypes(include=['object', 'string'])
print(non_numeric.columns)
print(len(non_numeric.columns))

[dtype('float64') dtype('int64')]
Index([], dtype='object')
0


# Step 5: Handling missing data

In [ ]:
#Check for features with more than 50% missing data
missing_percentage =  df_train.isnull().mean()
features_to_remove = missing_percentage[missing_percentage > 0.5].index.tolist()
print(features_to_remove)

#Remove features with more than 50% missing data
df_train= df_train.drop(columns = features_to_remove)
df_val = df_val.drop(columns = features_to_remove)
df_test = df_test.drop(columns = features_to_remove)



['MCQ230B', 'DMDYRSUS', 'MCQ050', 'PEASCCT1', 'BPXDI4', 'DMQADFC', 'MCQ170L', 'BPAEN4', 'MCQ206', 'MCQ170K', 'MCQ230A', 'BPXSY4', 'MCQ195', 'AGQ030', 'RIDEXPRG', 'MCQ170M', 'MCQ035', 'MCQ025', 'MCQ230C', 'MCQ040', 'MCD093']


In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3217, 113)
(1073, 113)
(1073, 113)


### Step 5a: Clean other categorical and numerical features that were not removed




In [ ]:
# Use unique value count instead of dtype to detect categorical vars
def cat_var_checker_by_uniques(df, threshold=20):
    return pd.DataFrame([
        [col, df[col].nunique(dropna=False)]
        for col in df.columns
        if df[col].nunique(dropna=False) <= threshold
    ], columns=['var', 'nunique']).sort_values(by='nunique', ascending=False)

# Get categorical columns (e.g. less than or equal to 20 unique values)
cat_vars_df = cat_var_checker_by_uniques(df_train, threshold=20)
categorical_cols = cat_vars_df['var'].tolist()

# Define numeric columns as the rest
numeric_cols = [col for col in df_train.columns if col not in categorical_cols]

# **Check columns manually after 7/17 meeting**

In [ ]:
#Check which are categorical and which are numerical
print(numeric_cols)
print(categorical_cols)

['LBXMPAH', 'LBDTCSI', 'SDMVSTRA', 'URXMNP', 'BPXDI1', 'LBXPFHS', 'WTSB2YR_x', 'LBXPFNA', 'RIDAGEYR', 'WTMEC2YR', 'INDFMPIR', 'URXCOP', 'URXMIB', 'URXMC1', 'LBXTC', 'URXMOH', 'URXECP', 'URXMHH', 'BPXML1', 'LBXPFUA', 'URXMZP', 'URXMBP', 'WTINT2YR', 'URXMEP', 'BPXPLS', 'LBDHDD', 'LBDHDDSI', 'URXMHNC', 'BPXSY3', 'BPXSY1', 'LBXPFDE', 'BPXSY2', 'BPXDI3', 'BPXDI2', 'WTSB2YR_y', 'URXMHP', 'URXCNP']
['INDFMIN2', 'INDHHIN2', 'DMDMARTL', 'DMDEDUC2', 'DMDFMSIZ', 'DMDHHSIZ', 'RIDRETH3', 'RIDRETH1', 'DMDCITZN', 'DMDHHSZB', 'BPACSZ', 'DMDHHSZA', 'MCQ092', 'MCQ300B', 'AIALANGA', 'MCQ203', 'DMDHHSZE', 'MCQ300A', 'MCQ053', 'MCQ010', 'URDCOPLC', 'LBDPFNAL', 'MCQ160G', 'MCQ160L', 'MCQ160O', 'LBDPFHSL', 'LBDPFUAL', 'BPAEN2', 'URDMNPLC', 'MCQ160A', 'MCQ080', 'BPAARM', 'LBDPFDEL', 'MCQ220', 'MCQ300C', 'URDMC1LC', 'URDMCHLC', 'URDECPLC', 'FIALANG', 'URDCNPLC', 'BPAEN1', 'MCQ160B', 'MCQ160N', 'BPAEN3', 'DMDBORN4', 'MCQ160D', 'LBDMPAHL', 'FIAPROXY', 'FIAINTRP', 'MCQ160E', 'MIALANG', 'URDMBPLC', 'URDMOHLC', 'UR

In [ ]:
#Check skew for imputation

skewness = df_train[numeric_cols].skew().sort_values()
# Normally distributed = skew between -1 and 1
mean_impute_cols = skewness[(-1 <= skewness) & (skewness <= 1)].index.tolist()
median_impute_cols = skewness[(skewness < -1) | (skewness > 1)].index.tolist()

In [ ]:
from sklearn.impute import SimpleImputer

# Mean imputation
mean_imputer = SimpleImputer(strategy='mean')
df_train[mean_impute_cols] = mean_imputer.fit_transform(df_train[mean_impute_cols])
df_val[mean_impute_cols] = mean_imputer.transform(df_val[mean_impute_cols])
df_test[mean_impute_cols] = mean_imputer.transform(df_test[mean_impute_cols])

# Median imputation
median_imputer = SimpleImputer(strategy='median')
df_train[median_impute_cols] = median_imputer.fit_transform(df_train[median_impute_cols])
df_val[median_impute_cols] = median_imputer.transform(df_val[median_impute_cols])
df_test[median_impute_cols] = median_imputer.transform(df_test[median_impute_cols])

In [ ]:
#Categorical imputation
mode_imputer = SimpleImputer(strategy='most_frequent')
df_train[categorical_cols] = mode_imputer.fit_transform(df_train[categorical_cols])
df_val[categorical_cols] = mode_imputer.transform(df_val[categorical_cols])
df_test[categorical_cols] = mode_imputer.transform(df_test[categorical_cols])

In [ ]:
#Check if imputation worked
print("Nulls in train:", df_train.isnull().sum().sum())
print("Nulls in val:", df_val.isnull().sum().sum())
print("Nulls in test:", df_test.isnull().sum().sum())

Nulls in train: 0
Nulls in val: 0
Nulls in test: 0


# Step 6: Encoding the data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Count unique values for each categorical variable
cat_unique_counts = df_train[categorical_cols].nunique().sort_values(ascending=False)
print(cat_unique_counts.tolist())
print(cat_unique_counts.sum())

[16, 16, 7, 7, 7, 7, 6, 5, 5, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
243


In [ ]:
#Set threshold for features that will be one hot encoded
ohe_threshold = 10

#Split categorical features by cardinality
low_card_cols = cat_unique_counts[cat_unique_counts < ohe_threshold].index.tolist()
high_card_cols = cat_unique_counts[cat_unique_counts >= ohe_threshold].index.tolist()

#Find features with just one unique value count
constant_cols = cat_unique_counts[cat_unique_counts == 1].index.tolist()

In [ ]:
#Drop features with just one value count
df_train.drop(columns=constant_cols, inplace=True)
df_val.drop(columns=constant_cols, inplace=True)
df_test.drop(columns=constant_cols, inplace=True)

categorical_cols = [col for col in categorical_cols if col not in constant_cols]

In [ ]:
#Frequency encode high cardinality columns
for col in high_card_cols:
    freq = df_train[col].value_counts(normalize=True)
    df_train[col] = df_train[col].map(freq)
    df_val[col] = df_val[col].map(freq)
    df_test[col] = df_test[col].map(freq)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Set up one-hot encoder using ColumnTransformer
ohe = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), low_card_cols)
    ],
    remainder='passthrough'  # Keep all other columns (numerical + freq-encoded) as-is
)

# Step 7: Splitting the feature and target

In [ ]:
#Explicit target
target = 'MCQ160C'

#Feature matrix
X_train = df_train.drop(columns=[target])
X_val = df_val.drop(columns=[target])
X_test = df_test.drop(columns=[target])

#Target vector
y_train = df_train[target].values
y_val = df_val[target].values
y_test = df_test[target].values


# Step 8: Scaling the data

In [ ]:
numeric_cols = [col for col in df_train.columns
                if col not in categorical_cols + [target]]
len(numeric_cols)

37

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Step 9: Handling class imbalance

In [ ]:
#Check distribution of target

from imblearn.over_sampling import SMOTE

# Set random seed for reproducibility
random_seed = 42
smote = SMOTE(random_state=random_seed)

# Apply SMOTE to training data only
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the new class distribution
from collections import Counter
print("Original y_train distribution:", Counter(y_train))
print("Resampled y_train distribution:", Counter(y_train_resampled))

Original y_train distribution: Counter({np.float64(0.0): 3084, np.float64(1.0): 133})
Resampled y_train distribution: Counter({np.float64(0.0): 3084, np.float64(1.0): 3084})


# Step 10: Set up ML grids

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

models = {
    'lr': LogisticRegression(class_weight='balanced', random_state=random_seed),
    'mlpc': MLPClassifier(early_stopping=True, random_state=random_seed),
    'rfc': RandomForestClassifier(class_weight='balanced', random_state=random_seed),
    'hgbc': HistGradientBoostingClassifier(random_state=random_seed)
}

#Set up pipes
pipes = {}
for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

param_grids = {}

#Logistic regression parameters
param_grids['lr'] = {'model__C': [0.01, 0.1, 1, 10]}

#Multilayer perceptron classifier
param_grids['mlpc'] ={
    'model__hidden_layer_sizes': [(50,), (100,), (50, 50)],  # Example: different hidden layer configurations
     'model__activation': ['relu', 'tanh'],
     'model__solver': ['adam', 'sgd'],
     'model__alpha': [0.0001, 0.001, 0.01],  # Regularization strength
     'model__learning_rate': ['constant', 'invscaling', 'adaptive']
     }


#Random forest
param_grids['rfc'] = {
     'model__n_estimators': [50, 100, 200],
     'model__max_depth': [10, 20, None],
     'model__min_samples_split': [2, 5, 10],
     'model__min_samples_leaf': [1, 2, 4],
     'model__bootstrap': [True, False]}


#Histogrambased gradient boosting
param_grids['hgbc'] = {
     'model__learning_rate': [0.01, 0.1, 0.5],
     'model__max_iter': [100, 200],
     'model__max_depth': [3, 5, 7],
     'model__min_samples_leaf': [20, 30, 50]}

# Step 11: Train and evaluate the models

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

grid_searches = {}
model_predictions = {}

for acronym, pipe in pipes.items():
    print(f"\nTraining model: {acronym.upper()}")

    # Use SMOTE for models without class_weight parameter
    if acronym in ['mlpc', 'hgbc']:
        # Apply SMOTE only on training data
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        grid_search = GridSearchCV(pipe, param_grid=param_grids[acronym],
                                   cv=5, n_jobs=-1, scoring='f1_macro', verbose=1)
        grid_search.fit(X_train_resampled, y_train_resampled)
    else:
        # Fit directly for models with class_weight (lr, rfc)
        grid_search = GridSearchCV(pipe, param_grid=param_grids[acronym],
                                   cv=5, n_jobs=-1, scoring='f1_macro', verbose=1)
        grid_search.fit(X_train, y_train)

    grid_searches[acronym] = grid_search

    print(f"Best Parameters for {acronym}: {grid_search.best_params_}")
    print(f"Best CV f1_macro Score: {grid_search.best_score_:.4f}")

    # Predict on test data
    y_pred = grid_search.best_estimator_.predict(X_test)
    model_predictions[acronym] = y_pred

    print(f"Prediction shape: {y_pred.shape} | True labels shape: {y_test.shape}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=["Class 0", "Class 1"]))



Training model: LR
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best Parameters for lr: {'model__C': 0.1}
Best CV f1_macro Score: 0.6482
Prediction shape: (1073,) | True labels shape: (1073,)
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.98      0.91      0.94      1029
     Class 1       0.22      0.57      0.31        44

    accuracy                           0.90      1073
   macro avg       0.60      0.74      0.63      1073
weighted avg       0.95      0.90      0.92      1073


Training model: MLPC
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [9]:
combined_df.to_csv("cleaned_combined_df.csv, index = False")